In [0]:
%pip install omegaconf
%restart_python

In [0]:

from omegaconf import OmegaConf
import os

REPO_ROOT = os.getcwd()
CFG_PATH = os.path.join(REPO_ROOT, "config", "config.yaml")

config = OmegaConf.load(CFG_PATH)

display({
    "env": config.env,
    "params": dict(config.params),
    "paths": dict(config.paths),
    "delivery_types": dict(config.delivery_types),
    "unit_factors": dict(config.unit_factors),
})


out_path = config.paths.output_root

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS RDV;
CREATE SCHEMA IF NOT EXISTS UDV;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS RDV.data_ventas (
  pais VARCHAR(2) ,
  fecha_proceso DATE  ,
  transporte STRING,
  ruta STRING ,
  tipo_entrega STRING,
  material STRING,
  precio DECIMAL(21,2) ,
  cantidad DECIMAL(21,2) COMMENT 'Cantidad segun la unidad' ,
  unidad VARCHAR(2) COMMENT 'Tipo de unidad (caja , unidad)'
)
USING DELTA
PARTITIONED BY (fecha_proceso)

In [0]:
%sql
--DROP TABLE UDV.data_ventas_depurado

In [0]:
%sql
--drop table UDV.data_ventas_obs

In [0]:
%sql
CREATE TABLE IF NOT EXISTS UDV.data_ventas_depurado (
  cod_pais VARCHAR(2) ,
  fec_proceso DATE  ,
  cod_transporte STRING,
  cod_ruta STRING ,
  cod_tipo_entrega STRING,
  cod_material STRING,
  precio_unitario_unidades DECIMAL(21,3) COMMENT 'Precio unitario por unidad' ,
  mto_venta DECIMAL(21,2) ,
  cant_uni_medida DECIMAL(21,2) COMMENT 'Cantidad segun la unidad de medida',
  cod_uni_medida VARCHAR(2) COMMENT 'Tipo de unidad (caja , unidad)',
  cant_unidades DECIMAL(21,2) COMMENT 'Cantidad en unidades',
  ind_rutina BOOLEAN COMMENT 'Indicador de rutina',
  ind_bonificacion BOOLEAN COMMENT 'Indicador de bonificación',
  origen_datos STRING COMMENT 'Archivo origen',
  fec_actualizacion_registro	STRING	COMMENT 'Fecha de actualización del registro (en formato string)'
)
USING DELTA
PARTITIONED BY (fec_proceso)
--LOCATION config.paths.output_root

;

In [0]:
%sql
CREATE TABLE IF NOT EXISTS UDV.data_ventas_obs (
  cod_pais STRING ,
  fec_proceso STRING  ,
  cod_transporte STRING,
  cod_ruta STRING ,
  cod_tipo_entrega STRING,
  cod_material STRING,
  mto_venta STRING ,
  cant_uni_medida STRING,
  cod_uni_medida STRING,
  motivo_obs STRING COMMENT 'Motivo de la observación',
  origen_datos STRING COMMENT 'Archivo origen',
  fec_actualizacion_registro	STRING	COMMENT 'Fecha de actualización del registro (en formato string)'
)
USING DELTA
PARTITIONED BY (fec_proceso);

### CAPA BRONZE:

In [0]:

from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import IntegerType,DoubleType,StringType

spark = SparkSession.builder.getOrCreate()


df_csv = (spark.read
          .option("header", True)
          .option("inferSchema", False)
          .csv(config.paths.raw_csv)) 
df_csv.limit(10).display()



In [0]:
df_ventas = df_csv.select(
    F.col("pais"),
    F.to_date(F.regexp_replace(F.col("fecha_proceso"), r"\s+", ""), "yyyyMMdd").alias("fecha_proceso"),
    F.col("transporte"),
    F.col("ruta"),
    F.col("tipo_entrega"),
    F.col("material"),
    F.col("precio").cast("decimal(21,2)"),
    F.col("cantidad").cast("decimal(21,2)"),
    F.col("unidad")
)

# FILTRADO DE LOS DATOS SEGUN EL ARCHIVO YAML
''' 
df_ventas = df_ventas.filter(
    (F.col("pais") == config.params.country) &
    (F.col("fecha_proceso") >= config.params.start_date) &
    (F.col("fecha_proceso") <= config.params.end_date)
)
''' 

df_ventas.limit(10).display()


In [0]:
df_ventas.write.format('delta').mode("overwrite").partitionBy("fecha_proceso").saveAsTable("RDV.data_ventas") 


### CAPA SILVER:

In [0]:
valid_rutina = [s.strip().upper() for s in config.delivery_types.routine]   
valid_bonif  = [s.strip().upper() for s in config.delivery_types.bonus]    
keys   = [F.lit(k.strip().upper()) for k in config.unit_factors.keys()]
vals   = [F.lit(float(v))          for v in config.unit_factors.values()]
factor_map = F.map_from_arrays(F.array(*keys), F.array(*vals))

In [0]:

df_rdv_ventas = spark.read.table("RDV.data_ventas")

df_rdv_ventas = df_rdv_ventas.select(
    F.col("pais").alias("cod_pais"),
    F.col("fecha_proceso").alias("fec_proceso"),
    F.col("transporte").alias("cod_transporte"),
    F.col("ruta").alias("cod_ruta"),
    F.col("tipo_entrega").alias("cod_tipo_entrega"),
    F.col("material").alias("cod_material"),
    F.col("precio").alias("mto_venta"),
    F.col("cantidad").alias("cant_uni_medida"),
    F.col("unidad").alias("cod_uni_medida")
)

df_rdv_ventas.limit(5).display()



In [0]:
factor_expr = F.element_at(
    factor_map, 
    F.upper(F.trim(F.col("cod_uni_medida")))
)

errores_concat = F.concat_ws(
    "|",
    F.when(F.col("cant_uni_medida").isNull(), F.lit("ERR_CANT_NULL")),
    F.when(F.col("cant_uni_medida").isNotNull() & (F.col("cant_uni_medida") <= 0),
           F.lit("ERR_CANT_NO_POSITIVA")),
    F.when(factor_expr.isNull(), F.lit("ERR_UNIDAD_DESCONOCIDA")),
    F.when(~F.upper(F.trim(F.col("cod_tipo_entrega"))).isin(valid_rutina + valid_bonif),
           F.lit("ERR_TIPO_ENTREGA_NO_CONSIDERADA"))
)

df_rdv_ventas = df_rdv_ventas.select(
    # claves/fechas
    F.col("cod_pais"),
    F.col("fec_proceso"),

    # llaves/códigos
    F.col("cod_transporte"),
    F.col("cod_ruta"),
    F.col("cod_tipo_entrega"),
    F.col("cod_material"),

    # precio unitario en UNIDADES (ST) = total / (cantidad_en_unidades)
    F.when(
        factor_expr.isNotNull() &
        F.col("cant_uni_medida").isNotNull() & (F.col("cant_uni_medida") > 0) &
        F.col("mto_venta").isNotNull(),
        F.round(
            F.col("mto_venta") / (F.col("cant_uni_medida") * factor_expr),
            3
        )
    ).cast("decimal(21,3)").alias("precio_unitario_unidades"),

    # monto de venta total (ya viene total por fila)
    F.round(F.col("mto_venta"), 2).cast("decimal(21,2)").alias("mto_venta"),

    # cantidad en unidad original
    F.round(F.col("cant_uni_medida"), 2).cast("decimal(21,2)").alias("cant_uni_medida"),

    # código de unidad original
    F.col("cod_uni_medida"),

    # cantidad convertida a UNIDADES (ST)
    F.when(
        factor_expr.isNotNull() &
        F.col("cant_uni_medida").isNotNull() & (F.col("cant_uni_medida") > 0),
        F.round(F.col("cant_uni_medida") * factor_expr, 2)
    ).cast("decimal(21,2)").alias("cant_unidades"),

    # indicadores booleanos
    F.when(F.upper(F.trim(F.col("cod_tipo_entrega"))).isin(valid_rutina), True)
     .when(F.upper(F.trim(F.col("cod_tipo_entrega"))).isin(valid_bonif), False)
     .otherwise(F.lit(None)).alias("ind_rutina"),

    F.when(F.upper(F.trim(F.col("cod_tipo_entrega"))).isin(valid_bonif), True)
     .when(F.upper(F.trim(F.col("cod_tipo_entrega"))).isin(valid_rutina), False)
     .otherwise(F.lit(None)).alias("ind_bonificacion"),

    # trazabilidad
    F.lit(config.paths.raw_csv).alias("origen_datos"),
    F.date_format(F.current_timestamp(), "yyyy-MM-dd HH:mm:ss").alias("fec_actualizacion_registro"),

    # motivo_obs (NULL si no hubo errores)
    F.when(F.length(errores_concat) == 0, F.lit(None)).otherwise(errores_concat).alias("motivo_obs")
)

df_depurado = df_rdv_ventas.filter(
    (F.col("motivo_obs").isNull()) & 
    F.col("ind_rutina").isNotNull() & 
    F.col("ind_bonificacion").isNotNull() 
).drop("motivo_obs")

df_depurado.write.format('delta').mode("overwrite").partitionBy("fec_proceso").saveAsTable("UDV.data_ventas_depurado") 


In [0]:

df_obs = df_rdv_ventas.select(
    F.col("cod_pais").cast("string"),
    F.col("fec_proceso").cast("string"),
    F.col("cod_transporte").cast("string"),
    F.col("cod_ruta").cast("string"),
    F.col("cod_tipo_entrega").cast("string"),
    F.col("cod_material").cast("string"),
    F.col("mto_venta").cast("string"),
    F.col("cant_uni_medida").cast("string"),
    F.col("cod_uni_medida").cast("string"),
    F.col("motivo_obs").cast("string"),
    F.col("origen_datos").cast("string"),
    F.col("fec_actualizacion_registro").cast("string")
).filter(
    F.col("motivo_obs").isNotNull()
)

df_obs.write.format('delta').mode("overwrite").partitionBy("fec_proceso").save(out_path)